# Calculating travel time to ID registration sites in Liberia

As part of a project supporting a national ID system in Liberia we are calculating travel time to physical ID sites.

The country has limited financial capacity, so they are looking for as efficient a solution as possible. At the same time, there needs to be a focus on equality - we cannot miss just the people who are hard to reach

Goal is to prioritize investment in existing licensing centers, and provide some metrics to identify which proposed centers should be supported:
## Existing licensing centers
1.	How many people within 60 mins and 120 mins drive time  
    a. Summarize at ADM1 and ADM2
2.	How many exclusive users within 120 mins drive time (Marketsheds)
3.	ADM1 and ADM2 summaries of population outside 120 min drive time of existing centers  
    a. This population layer is labelled “unserved population” and used for several analyses below
## Proposed centers
1.	Total # of people within 60, 120 min vectors  
    a. Replicate using the population layer from #3 above, which is the unserved population
2.	Marketsheds  
    a. Replicate using unserved population layer
3.	Prioritize proposed centers sequentially using the results of #1 and #2 to identify how to serve the most population with the fewest centers


In [ ]:
import sys
import os
import rasterio
import overturemaps

import pandas as pd
import geopandas as gpd
import numpy as np
import skimage.graph as graph

from shapely.geometry import Point, box
# from space2stats_client import Space2StatsClient

### TODO: Update GOSTrocks and GOSTnetsraster imports to use package structure
sys.path.insert(0, r"C:\WBG\Work\Code\GOSTrocks\src")
import GOSTrocks.rasterMisc as rMisc
import GOSTrocks.mapMisc as mapMisc
from GOSTrocks.misc import tPrint

sys.path.append(r"C:\WBG\Work\Code\GOSTnetsraster\src")
import GOSTnetsraster.market_access as ma
import GOSTnetsraster.conversion_tables as speed_tables

%load_ext autoreload
%autoreload 2

In [ ]:
# Input parameters
m_crs = "ESRI:54009"  # Need to project data to a metres-based projection
ISO3 = "LBR"

# Define input data
base_folder = "C:/WBG/Work/Projects/LBR_Road_Improvements"
for c_folder in [
    os.path.join(base_folder, "DATA"),
    os.path.join(base_folder, "RESULTS"),
]:
    if not os.path.exists(c_folder):
        os.makedirs(c_folder)
landcover_file = os.path.join(base_folder, "DATA", "ESA_Globcover.tif")
# These are the digitized road segments that have been improved

transport_network = os.path.join(
    base_folder, "DATA", "Overture", "transport_network.gpkg"
)
major_roads_file = os.path.join(base_folder, "DATA", "Overture", "major_roads.gpkg")
fixed_id_sites_file = os.path.join(base_folder, "DATA", "LBR_ID_sites.kmz")
proposed_id_sites_file = os.path.join(
    base_folder, "DATA", "Enrollment_Locations_phase1_tentative.csv"
)

# WorldPop 2020 constrained, projected to m_crs
pop_file = os.path.join(base_folder, "DATA", "lbr_pop_2025_CN_1km_R2025A_UA_v1.tif")
pop_file_synced = os.path.join(
    base_folder, "DATA", "lbr_pop_2025_CN_1km_R2025A_UA_v1_synced.tif"
)

# administrative boundaries are used to summarize population
global_adm2_file = (
    r"C:\WBG\Work\data\ADMIN\NEW_WB_BOUNDS\FOR_PUBLICATION\geojson\WB_GAD_ADM2.geojson"
)
adm2 = gpd.read_file(global_adm2_file)
adm2 = adm2.loc[adm2["ISO_A3"] == ISO3]

global_adm0_file = (
    r"C:\WBG\Work\data\ADMIN\NEW_WB_BOUNDS\FOR_PUBLICATION\geojson\WB_GAD_ADM0.geojson"
)
adm0 = gpd.read_file(global_adm0_file)

# Define output files
friction_folder = os.path.join(base_folder, "DATA", "FRICTION")
results_folder = os.path.join(base_folder, "RESULTS")
overture_folder = os.path.join(base_folder, "DATA", "Overture")
for cFolder in [friction_folder, results_folder, overture_folder]:
    if not os.path.exists(cFolder):
        os.makedirs(cFolder)

friction_file = os.path.join(friction_folder, "GLOBAL_FRICTION_{}.tif".format(ISO3))
friction_file_proj = os.path.join(
    friction_folder, "GLOBAL_FRICTION_{}_proj.tif".format(ISO3)
)

In [ ]:
# Download roads from Overture
# Download transport network
if not os.path.exists(transport_network):
    bbox = adm2.total_bounds.tolist()  # minx, miny, maxx, maxy
    transport = overturemaps.record_batch_reader("segment", bbox).read_all()
    transport_df = gpd.GeoDataFrame.from_arrow(transport)
    transport_df.crs = 4326
    transport_df.loc[
        :,
        [
            "id",
            "class",
            "subtype",
            "road_surface",
            "speed_limits",
            "width_rules",
            "geometry",
        ],
    ].to_file(transport_network, driver="GPKG")

# process transport to a) remove roads outside IRAQ and b) remove all roads of OSMLR class 3 and 4
if not os.path.exists(major_roads_file):
    roads = gpd.read_file(transport_network)
    roads["OSMLR_class"] = roads["class"].map(speed_tables.OSMLR_Classes)
    roads_joined = gpd.sjoin(roads, adm2, how="inner", predicate="intersects")
    major_roads = roads_joined.loc[
        roads_joined["OSMLR_class"].isin(["OSMLR level 1", "OSMLR level 2"]),
        roads.columns,
    ]
    major_roads.to_file(major_roads_file, driver="GPKG", index=False)

In [ ]:
# Extract friction surface for country
if not os.path.exists(friction_file_proj):
    global_friction_file = (
        r"C:\WBG\Work\data\FRICTION\2020_motorized_friction_surface.geotiff"
    )
    rMisc.clipRaster(
        rasterio.open(global_friction_file),
        adm2.to_crs(rasterio.open(global_friction_file).crs),
        friction_file,
        crop=False,
    )
    rMisc.project_raster(rasterio.open(friction_file), m_crs, friction_file_proj)

# Sync the population raster to the friction layer
if not os.path.exists(pop_file_synced):
    rMisc.standardizeInputRasters(
        rasterio.open(pop_file),
        rasterio.open(friction_file_proj),
        pop_file_synced,
        resampling_type="sum",
    )

# Map road network and service centers

In [ ]:
major_roads = gpd.read_file(major_roads_file)
major_roads["map_class"] = major_roads["OSMLR_class"].apply(lambda x: 5 - int(x[-1]))
lbr_country = adm0.loc[adm0["ISO_A3"] == "LBR"]
other_countries = adm0.loc[adm0["ISO_A3"] != "LBR"]

In [ ]:
proposed_sites = pd.read_csv(proposed_id_sites_file)
proposed_sites["geometry"] = proposed_sites.apply(
    lambda row: Point(row["X_UTM"], row["Y_UTM"]), axis=1
)
proposed_sites = gpd.GeoDataFrame(proposed_sites, geometry="geometry", crs=10802)
proposed_sites = proposed_sites.to_crs(major_roads.crs)

existing_sites = gpd.read_file(fixed_id_sites_file, driver="KML")
existing_sites = existing_sites.to_crs(major_roads.crs)

In [ ]:
plt, fig, ax = mapMisc.static_map_vector(
    major_roads,
    "map_class",
    colormap="Dark2",
    add_basemap=False,
    add_wb_borders_lines=False,
)

lbr_country.plot(ax=ax, facecolor="white", edgecolor="black", linewidth=1, zorder=0)
other_countries.plot(
    ax=ax, facecolor="lightgrey", edgecolor="black", linewidth=0.5, zorder=1
)
proposed_sites.plot(ax=ax, color="grey", markersize=20, marker="*")
existing_sites.plot(ax=ax, color="red", markersize=40, marker="o")
bbox = lbr_country.total_bounds
ax.set_xlim(bbox[0], bbox[2])
ax.set_ylim(bbox[1], bbox[3])
ax = ax.set_axis_off()
plt.title("LBR Major Roads and Service Centers", fontsize=16)
plt.savefig(
    "maps/LBR_existing_and_proposed_service_centers.png", dpi=300, bbox_inches="tight"
)
plt.show()

## Calculate travel time to ID registration sites

In [ ]:
in_pop = rasterio.open(pop_file_synced)
pre_friction = rasterio.open(friction_file_proj)
# Calculate pre-intervention, population-weighted travel times summarized at admin 2
frictionD = pre_friction.read()[0, :, :]
frictionD = frictionD * pre_friction.res[0]
mcp = graph.MCP_Geometric(frictionD)

drive_time_thresholds = [30, 60, 120, 180, 240]  # minutes


for dest, label in [(existing_sites, "existing_ID"), (proposed_sites, "proposed_ID")]:
    dest = dest.to_crs(pre_friction.crs)

    tt_file = os.path.join(results_folder, f"Travel_time_to_{label}.tif")
    tt_dest = ma.summarize_travel_time_populations(
        in_pop, pre_friction, dest, mcp, adm2, out_tt_file=tt_file
    )

    drive_vectors = ma.generate_feature_vectors(
        pre_friction, mcp, dest, drive_time_thresholds
    )

    # Summarize population within drive time thresholds
    dt_pop_summary = rMisc.zonalStats(drive_vectors, in_pop, return_df=True, minVal=0)
    dt_pop_summary["tempID"] = drive_vectors["IDX"]
    dt_pop_summary["threshold"] = drive_vectors["threshold"]

    population_summaries = pd.pivot_table(
        dt_pop_summary,
        values="SUM",
        index=["tempID"],
        columns=["threshold"],
        aggfunc="sum",
    ).reset_index()
    summarized_population_drivetime = pd.merge(dest, population_summaries, on="tempID")

    # Generate marketsheds and summarize population within each marketshed
    market_sheds = ma.generate_market_sheds(pre_friction, dest, column_id="tempID")
    pop_data = in_pop.read()[0, :, :]
    unq_dests = np.unique(market_sheds)
    tt_data = rasterio.open(tt_file).read()[0, :, :]

    out_res = {}
    for c_dest in unq_dests:
        dest_mask = market_sheds == c_dest
        dest_pop_sum = pop_data[dest_mask].sum()
        dest_pop_120min = pop_data[(dest_mask) & (tt_data <= 120)].sum()
        out_res[c_dest] = {
            "total_pop": dest_pop_sum,
            "pop_120min": dest_pop_120min,
            "n_pixels": dest_mask.sum(),
        }

    res = pd.DataFrame(out_res).T
    res["tempID"] = res.index
    final_marketsheds = pd.merge(dest, res, on="tempID")
    final_marketsheds.drop(["geometry", "tempID"], axis=1).to_csv(
        os.path.join(results_folder, f"Marketshed_population_{label}.csv")
    )

    # Write geospatial data to file
    tt_dest.to_file(
        os.path.join(results_folder, f"ADM2_tt_{label}.gpkg"), driver="GPKG"
    )
    drive_vectors.to_file(
        os.path.join(results_folder, f"Drive_time_vectors_{label}.gpkg"), driver="GPKG"
    )
    with rasterio.open(
        os.path.join(results_folder, f"Marketsheds_{label}.tif"), "w", **in_pop.meta
    ) as out_raster:
        out_raster.write(market_sheds, 1)

    # Write tabular data to file
    pd.DataFrame(tt_dest.drop(["geometry"], axis=1)).to_csv(
        os.path.join(results_folder, f"ADM2_tt_{label}.csv")
    )
    summarized_population_drivetime.drop(["geometry", "tempID"], axis=1).to_csv(
        os.path.join(results_folder, f"Population_within_drivetime_{label}.csv")
    )

In [ ]:
# Prioritize proposed ID sites based on population within 60 driving minutes

drive_time_threshold = 60
dests = proposed_sites
dests = dests.to_crs(pre_friction.crs)
drive_vectors = ma.generate_feature_vectors(
    pre_friction, mcp, dests, [drive_time_threshold]
)

pop_R = rasterio.open(pop_file_synced)
pop_data = pop_R.read()[0, :, :]
tt_data = rasterio.open(
    os.path.join(results_folder, "Travel_time_to_existing_ID.tif")
).read()[0, :, :]
temp_dt = drive_vectors.copy()
good_proposed_sites = []
idx = 0
while len(good_proposed_sites) < 100:
    idx += 1
    # Calculate the unserved population based on current existing sites
    unserved_tt_area = tt_data > drive_time_threshold
    unserved_pop = pop_data * unserved_tt_area
    with rMisc.create_rasterio_inmemory(
        pop_R.meta, unserved_pop
    ) as unserved_pop_raster:
        # Summarize population within drive time threshold for each proposed site
        dt_pop_summary = rMisc.zonalStats(
            temp_dt, unserved_pop_raster, return_df=True, minVal=0
        )
        dt_pop_summary["tempID"] = temp_dt["IDX"].values
        # Identify the proposed site that serves the maximum unserved population
        good_site_id = dt_pop_summary.sort_values(by="SUM", ascending=False).iloc[0][
            "tempID"
        ]
        good_site = dests.loc[dests["tempID"] == good_site_id]
        good_site["rank"] = idx
        good_proposed_sites.append(good_site)
        # Calculate drive time raster for the new good site
        tt_good, traceback_good = ma.calculate_travel_time(pop_R, mcp, good_site)
        # Update unserved travel time area
        unserved_tt_area = (unserved_tt_area) & (tt_good > drive_time_threshold)
        # Remove the selected site from the next calculation
        temp_dt.drop(temp_dt[temp_dt["IDX"] == good_site_id].index, inplace=True)
    tPrint(f"Selected {len(good_proposed_sites)} proposed sites so far.")

pd.concat(good_proposed_sites).to_file(
    os.path.join(results_folder, "Top_100_proposed_ID_sites.gpkg"), driver="GPKG"
)
pd.concat(good_proposed_sites).drop(["geometry"], axis=1).to_csv(
    os.path.join(results_folder, "Top_100_proposed_ID_sites.csv")
)

# Map results


In [ ]:
pd.set_option("mode.chained_assignment", None)
drive_vectors = gpd.read_file(
    os.path.join(results_folder, "Drive_time_vectors_existing_ID.gpkg")
)
plt, fig, ax = mapMisc.static_map_vector(
    drive_vectors.loc[drive_vectors["threshold"] == 60],
    "IDX",
    # thresh = list(range(0, len(drive_vectors.loc[drive_vectors['threshold'] == 60]))),
    add_basemap=False,
    add_wb_borders_lines=False,
)
existing_sites.to_crs(in_pop.crs).plot(ax=ax, color="grey", markersize=20, marker="*")

bbox = in_pop.bounds
# plot ocean
ocean = gpd.GeoDataFrame(
    geometry=[box(*bbox).difference(lbr_country.to_crs(in_pop.crs).union_all())],
    crs=in_pop.crs,
)
ocean.plot(ax=ax, color="midnightblue", zorder=0)
lbr_country.to_crs(in_pop.crs).plot(
    ax=ax, facecolor="white", edgecolor="black", linewidth=1, zorder=0
)
other_countries.to_crs(in_pop.crs).plot(
    ax=ax, facecolor="lightgrey", edgecolor="black", linewidth=0.5, zorder=1
)


ax.set_xlim(bbox[0], bbox[2])
ax.set_ylim(bbox[1], bbox[3])
ax = ax.set_axis_off()
plt.title("60 min drive-time existing ID centers", fontsize=16)
plt.savefig("maps/LBR_tt_vector_existing_ID.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Map tt to existing ID centers
tt_rast = rasterio.open(os.path.join(results_folder, "Travel_time_to_existing_ID.tif"))
plt, fig, ax = mapMisc.static_map_raster(
    tt_rast, reverse_colormap=True, thresh=[-1, 30, 60, 120, 180, 240]
)
existing_sites.to_crs(tt_rast.crs).plot(ax=ax, color="grey", markersize=20, marker="*")
other_countries.to_crs(tt_rast.crs).plot(
    ax=ax, facecolor="lightgrey", edgecolor="black", linewidth=0.5, zorder=1
)
bbox = tt_rast.bounds
# plot ocean
ocean = gpd.GeoDataFrame(
    geometry=[box(*bbox).difference(lbr_country.to_crs(tt_rast.crs).union_all())],
    crs=tt_rast.crs,
)
ocean.plot(ax=ax, color="midnightblue", zorder=0)
ax.set_xlim(bbox[0], bbox[2])
ax.set_ylim(bbox[1], bbox[3])
ax = ax.set_axis_off()
plt.title("Travel time to existing ID centers", fontsize=16)
plt.savefig("maps/LBR_tt_raster_existing_ID.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
with rMisc.create_rasterio_inmemory(in_pop.meta, market_sheds) as marketsR:
    plt, fig, ax = mapMisc.static_map_raster(marketsR)

other_countries.to_crs(in_pop.crs).plot(
    ax=ax, facecolor="lightgrey", edgecolor="black", linewidth=0.5, zorder=1
)
bbox = in_pop.bounds
# plot ocean
ocean = gpd.GeoDataFrame(
    geometry=[box(*bbox).difference(lbr_country.to_crs(in_pop.crs).union_all())],
    crs=in_pop.crs,
)
ocean.plot(ax=ax, color="midnightblue", zorder=0)
ax.set_xlim(bbox[0], bbox[2])
ax.set_ylim(bbox[1], bbox[3])
ax = ax.set_axis_off()
plt.title("LBR Market Sheds existing ID centers", fontsize=16)
plt.savefig("maps/LBR_market_sheds_existing_ID.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Map proposed top 100 sites alonog with existing sites
top_100_sites = gpd.read_file(
    os.path.join(results_folder, "Top_100_proposed_ID_sites.gpkg")
)
plt, fig, ax = mapMisc.static_map_vector(
    top_100_sites.to_crs(major_roads.crs),
    "rank",
    colormap="magma",
    reverse_colormap=True,
    add_basemap=False,
    add_wb_borders_lines=False,
)

major_roads.plot(ax=ax, color="lightgrey", linewidth=0.5)
lbr_country.plot(ax=ax, facecolor="white", edgecolor="black", linewidth=1, zorder=0)
other_countries.plot(
    ax=ax, facecolor="lightgrey", edgecolor="black", linewidth=0.5, zorder=1
)
existing_sites.plot(ax=ax, color="red", markersize=20, marker="o")

bbox = lbr_country.total_bounds
ax.set_xlim(bbox[0], bbox[2])
ax.set_ylim(bbox[1], bbox[3])
ax = ax.set_axis_off()
plt.title("LBR proposed top 100 sites and existing sites", fontsize=16)
plt.savefig(
    "maps/LBR_proposed_top_100_and_existing_service_centers.png",
    dpi=300,
    bbox_inches="tight",
)
plt.show()

# Write summary to readme

## Existing licensing centers
1.	How many people within 60 mins and 120 mins drive time  
    a. Summarize at ADM1 and ADM2
2.	How many exclusive users within 120 mins drive time (Marketsheds)
3.	ADM1 and ADM2 summaries of population outside 120 min drive time of existing centers  
    a. This population layer is labelled “unserved population” and used for several analyses below
## Proposed centers
1.	Total # of people within 60, 120 min vectors  
    a. Replicate using the population layer from #3 above, which is the unserved population
2.	Marketsheds  
    a. Replicate using unserved population layer
3.	Prioritize proposed centers sequentially using the results of #1 and #2 to identify how to serve the most population with the fewest centers

In [ ]:
# Write a README.md file summarizing the work so far
readme_file = "README.md"
with open(readme_file, "w") as f:
    f.write("# Liberia ID centre travel time analysis\n")
    f.write(
        "This analysis evaluates travel time to existing and proposed ID centers in Liberia.\n"
    )
    f.write(
        "<img src='maps/LBR_existing_and_proposed_service_centers.png' alt='Map of primary roads and service centers' width='600'/>\n"
    )
    f.write("\n")

    f.write(
        "This was done using friction surface analysis to measure travel time to service centers, and map marketsheds for service centers\n"
    )

    f.write(
        "The following analysis were undertaken to quantify accessibility to ID centers in Liberia:\n"
    )
    f.write("| Question | Output file |\n")
    f.write("|---|---|\n")
    f.write(
        "| How many people are within 60 mins and 120 mins drive time to existing ID centers, summarized at ADM1 and ADM2 levels? | ADM2_tt_existing_ID.csv |\n"
    )
    f.write(
        "| How many exclusive users are within 120 mins drive time to existing ID centers (Marketsheds)? | Marketshed_population_existing_ID.csv |\n"
    )
    f.write(
        "| What is the population outside 120 min drive time of existing centers (unserved population)? | ADM2_tt_existing_ID.csv |\n"
    )
    f.write(
        "| How many people are within 60, 120 min drive time to proposed centers? | ADM2_tt_proposed_ID.csv |\n"
    )
    f.write(
        "| What are the Marketsheds for proposed centers? | Marketshed_population_proposed_ID.csv |\n"
    )
    f.write(
        "| How to prioritize proposed centers sequentially to serve the most population with the fewest centers? | Top_100_proposed_ID_sites.gpkg |\n"
    )
    f.write("\n")

    f.write("## Maps of results\n")
    f.write(
        "<img src='maps/LBR_tt_raster_existing_ID.png' alt='Travel time to existing ID centers map' width='600'/>\n"
    )
    f.write("\n")
    f.write(
        "The friction surface analysis measures travel time to the nearest destination from every location in the country, accounting for road networks and landcover-based travel speeds.\n\n"
    )
    f.write(
        "<img src='maps/LBR_tt_vector_existing_ID.png' alt='60 min drive-time existing ID centers' width='600'/>\n\n"
    )
    f.write(
        "Drive-time vectors show areas that can reach a service center within a given time threshold. This differs from the above analysis in that \
            each destination (ID centre) is treated independently, so the drive areas can overlap\n\n"
    )
    f.write(
        "<img src='maps/LBR_market_sheds_existing_ID.png' alt='LBR Market Sheds existing ID centers' width='600'/>\n\n"
    )
    f.write(
        "Marketsheds show the exclusive service area for each service center, i.e., the area that is closest to a given center compared to all other centers.\n"
    )
    f.write(
        "<img src='maps/LBR_proposed_top_100_and_existing_service_centers.png' alt='Map of proposed top 100 sites and existing sites' width='600'/>\n\n"
    )
    f.write(
        "The proposed top 100 sites are selected sequentially to maximize the unserved population within 60 mins drive time.\n"
    )
    f.write("\n")